In [177]:
import re
import pandas as pd

from PyPDF2 import PdfReader
from datetime import datetime

In [2]:
project_path = r'C:\Users\robsc\Documents\Python\Huishoudboekje'
filename = 'data/Rabobank/NL45RABO0372430163-EUR-2022-04.pdf'

In [3]:
reader = PdfReader(project_path + '/' + filename)
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()

In [4]:
def get_header(text):
    
    string_name = 'Bedrag bij (credit)\n'
    
    header_num = text.find(string_name)
    
    return text[:(header_num + len(string_name))]

In [5]:
header = get_header(text)

In [6]:
def string_to_value(string):
    
    return float(string.split(' ')[0].replace('.', '').replace(',', '.'))

In [7]:
def get_account_values(text):
    """Only for page 1"""
    
    account_values = ['Beginsaldo', 'Eindsaldo', 'Totaal afgeschreven', 'Totaal bijgeschreven']
    dict_account = {}
    
    for num, val in enumerate(text):

        if val in account_values:
            dict_account[val] = string_to_value(text[num+1])
            
    return dict_account

In [8]:
dict_account = get_account_values(header.split('\n'))

In [9]:
def get_body(text, header):
    
    return text.replace('\nCR = tegoed\nD   = tekort\n', '').replace(header, '')

In [195]:
body = get_body(text, header)

In [196]:
body

'01-04\nei\nNL24 RABO 0168 4762 07\n21,66\nBasic Fit Nederland B.V.\n01-04-2022 / 30-04-2022\nKenmerk machtiging / incassant ID:\n305012750\nNL47ZZZ370924590000\nTransactiereferentie:\nNO305012750-130\nVerwerkingsdatum: 01-04-2022\n01-04\nei\nNL02 RABO 0387 5120 12\n10,00\nZOA\nHartelijk dank voor uw bijdrage.\nKenmerk machtiging / incassant ID:\nZOA201706557823\nNL39ZZZ410097230000\nTransactiereferentie:\n3022041000628634\nVerwerkingsdatum: 01-04-2022\n01-04\nbc\nTango Amersfoort 1 AMERSFOORT\n84,40\nBetaalautomaat 08:45 pasnr. 038\nApple Pay\nVerwerkingsdatum: 01-04-2022\n01-04\nbc\nHoogvliet 700 5 AMERSFOORT\n28,93\nBetaalautomaat 09:07 pasnr. 038\nApple Pay\nVerwerkingsdatum: 01-04-2022\n01-04\nei\nNL15 DEUT 0319 8719 75\n17,27\nNationale-Nederlanden Verzekeren\nINCASSO ZEKERHEIDSPAKKET PARTICULIE\nREN POLIS 154044168025 PERIODE 01-0\n4-2022 TOT 01-05-2022 UW TP NATIONA\nLE-NEDERLANDEN\nKenmerk machtiging / incassant ID:\n4379705V1\nNL34ZZZ664133540000\nTransactiereferentie:\n51505

In [12]:
def get_all_transactions(reader):
    
    num_of_pages = len(reader.pages)
    
    list_bodies = []
    
    for num, page in enumerate(reader.pages):
        
        text = page.extract_text()
        
        header = get_header(text)
        
        if num == 0:
            dict_account = get_account_values(header.split('\n'))
            
        body = get_body(text, header)
        
        list_bodies.append(body)
        
    all_trans = '\n'.join(list_bodies)
    
    return all_trans[:all_trans.find('\nac = acceptgiro')]

In [13]:
trans = get_all_transactions(reader)

In [42]:
trans_new = ' '.join(trans.split('\n'))

In [100]:
matches = re.findall('[0-9][0-9]-[0-9][0-9] [a-z][a-z].+?(?=Verwerkingsdatum: [0-9][0-9]-[0-9][0-9]-202[0-9])', trans_new)

In [191]:
def get_info_iban(match):
    
    if re.search('[A-Z][A-Z][0-9][0-9] [0-9][0-9][0-9][0-9] [0-9][0-9][0-9][0-9] [0-9][0-9][0-9][0-9] [0-9][0-9][0-9][0-9] [0-9][0-9] ', match):
        print(match)
        amount = match.split()[8]

        incassant = ' '.join(match.split()[9:12])
        
    else:

        amount = match.split()[7]
        incassant = ' '.join(match.split()[8:11])
    
    return amount, incassant

def get_info_bc(match):
    
    rgx = re.search('([^\s]*?),[0-9][0-9]', match)

    amount = rgx.group(0)

    incassant = match[9:(rgx.span(0)[0]-1)]
    
    return amount, incassant


In [192]:
## transactions with tb can be ignored? tb = eigen rekening => geen uitgaven
## ba = betaalautomaat => this time credit, but probably not always

dict_trans = {}

for no, match in enumerate(matches):
    
    print(f'Transaction #{no}')

    date = datetime(2022, int(match[3:5]), int(match[:2]))
    trans_type = match[6:8]
    
    if trans_type in ['ei', 'id', 'cb', 'sb', 'bg', 'tb']:
        amount, incassant = get_info_iban(match)
    elif trans_type in ['bc', 'db', 'cc', 'ba']:
        amount, incassant = get_info_bc(match)
    else:
        raise ValueError(f'Unknown transaction type={trans_type}')
        
    if trans_type in ['cb', 'sb', 'tb', 'ba']:
        fin_type = 'credit'
    else:
        fin_type = 'debet'
        
    try:
        amount_f = float(amount.replace('.', '').replace(',', '.'))
    except:
        raise ValueError(f'Unable to convert to float={amount}')
        
    dict_trans[no] = {
        'DATE': date, 'TRANSACTION_TYPE': trans_type, 'FINANCIAL_TYPE': fin_type, 'PARTY': incassant, 'AMOUNT': amount_f
    }
    
df = pd.DataFrame.from_dict(dict_trans, orient='index')

print(df)

Transaction #0
Transaction #1
Transaction #2
Transaction #3
Transaction #4
Transaction #5
Transaction #6
Transaction #7
Transaction #8
Transaction #9
Transaction #10
Transaction #11
Transaction #12
Transaction #13
Transaction #14
Transaction #15
Transaction #16
Transaction #17
Transaction #18
Transaction #19
Transaction #20
Transaction #21
Transaction #22
Transaction #23
Transaction #24
Transaction #25
Transaction #26
Transaction #27
Transaction #28
Transaction #29
Transaction #30
Transaction #31
Transaction #32
Transaction #33
Transaction #34
Transaction #35
11-04 id DE54 2022 0800 0093 1032 90 5,00 Carshare Ventures BV 25NC2KJ 0140000421114125 0 - Verifi cation Snappcar x0163 pasnr.038 Transactiereferentie: 11-04-2022 09:27 0140000421114125 
Transaction #36
Transaction #37
Transaction #38
Transaction #39
Transaction #40
Transaction #41
Transaction #42
Transaction #43
Transaction #44
Transaction #45
Transaction #46
Transaction #47
Transaction #48
Transaction #49
Transaction #50
Transa

In [193]:
df.groupby('FINANCIAL_TYPE')['AMOUNT'].sum()

FINANCIAL_TYPE
credit     9140.62
debet     15438.89
Name: AMOUNT, dtype: float64

In [194]:
dict_account

{'Beginsaldo': 11997.2,
 'Eindsaldo': 5698.93,
 'Totaal afgeschreven': 15438.89,
 'Totaal bijgeschreven': 9140.62}

In [184]:
df.to_excel(r'C:\Users\robsc\Documents\Python\Huishoudboekje\data\Rabobank\output_april.xlsx')

In [111]:
## ei => IBAN

match = matches[0]

date = match[:5]
trans_type = match[6:8]

if re.search('NL[0-9][0-9] [A-Z][A-Z][A-Z][A-Z] [0-9][0-9][0-9][0-9] [0-9][0-9][0-9][0-9] [0-9][0-9]', match):
    print('IBAN found')
    
    amount = match.split()[7]
    
    incassant = ' '.join(match.split()[8:10])
print(date)
print(trans_type)
print(amount)
print(incassant)

IBAN found
01-04
ei
21,66
Basic Fit


In [113]:
## ei => IBAN

match = matches[1]

date = match[:5]
trans_type = match[6:8]

if re.search('NL[0-9][0-9] [A-Z][A-Z][A-Z][A-Z] [0-9][0-9][0-9][0-9] [0-9][0-9][0-9][0-9] [0-9][0-9]', match):
    print('IBAN found')
    
    amount = match.split()[7]
    
    incassant = match.split()[8]
print(date)
print(trans_type)
print(amount)
print(incassant)

IBAN found
01-04
ei
10,00
ZOA


In [130]:
## transaction type is bc

match = matches[2]

date = match[:5]
trans_type = match[6:8]

re_amount = re.search('([^\s]*?),[0-9][0-9]', match)

amount = re_amount.group(0)

incassant = match[9:(re_amount.span(0)[0]-1)] # first word would be sufficient

print(date)
print(trans_type)
print(amount)
print(incassant)

01-04
bc
84,40
Tango Amersfoort 1 AMERSFOORT


In [134]:
## transaction type is bc

match = matches[3]

date = match[:5]
trans_type = match[6:8]

re_amount = re.search('([^\s]*?),[0-9][0-9]', match)

amount = re_amount.group(0)

incassant = match[9:(re_amount.span(0)[0]-1)] # first word would be sufficient

print(date)
print(trans_type)
print(amount)
print(incassant)

01-04
bc
28,93
Hoogvliet 700 5 AMERSFOORT


In [136]:
## transaction type is ei

match = matches[4]

date = match[:5]
trans_type = match[6:8]

if re.search('[A-Z][A-Z][0-9][0-9] [0-9][0-9][0-9][0-9] [0-9][0-9][0-9][0-9] [0-9][0-9][0-9][0-9] [0-9][0-9][0-9][0-9] [0-9][0-9]', match):
    
    amount = match.split()[8]
    
    incassant = ' '.join(match.split()[9:12])

if re.search('NL[0-9][0-9] [A-Z][A-Z][A-Z][A-Z] [0-9][0-9][0-9][0-9] [0-9][0-9][0-9][0-9] [0-9][0-9]', match):
    print('IBAN found')
    
    amount = match.split()[7]
    
    incassant = ' '.join(match.split()[8:11])
print(date)
print(trans_type)
print(amount)
print(incassant)

IBAN found
01-04
ei
17,27
Nationale-Nederlanden


In [156]:
start = re.search('NL[0-9][0-9] [A-Z][A-Z][A-Z][A-Z] [0-9][0-9][0-9][0-9] [0-9][0-9][0-9][0-9] [0-9][0-9]', matches[8])
end = re.search('([^\s]*?),[0-9][0-9]', matches[8])

In [159]:
matches[16]

'04-04 sb NL97 INGB 0000 1138 98 57,00 Centrale Salarisadministratie Personeelsnummer: 2635173 Declarati e Transactiereferentie: 2635173-20220403-22012 '

In [197]:
from os import walk

In [201]:
filenames = next(walk(project_path + '/data/Rabobank'), (None, None, []))[2]

In [202]:
filenames

['NL45RABO0372430163-EUR-2022-04.pdf']